In [ ]:
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn import model_selection

Mounted at /content/drive


In [ ]:
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.35.197.226:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1387322569101108182),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, -4271433558254527203),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, -2702295881910584540),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, -1018064070307114394),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3947570167131390569),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, -7128575080053306105),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, -7040212790462838741),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, -5386975876186348188),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 171798691

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.35.197.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.35.197.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
class Conv1D:
  def __init__(self):
    
    self.encoder = LabelEncoder()
    self.summary = []

  def read_dataset(self,path):
    df = pd.read_csv(path)
    if 'USE_embed' in list(df.columns):
      df = df.drop('Unnamed: 0',axis = 1)
      df = df.rename({'USE_embed':'embeddings'},axis = 1)
    elif 'ELEC_embed' in list(df.columns):
      df = df.rename({'ELEC_embed':'embeddings'},axis = 1)
    
    d = {1:'VNeg',2:'Neg',3:'Neu',4:'Pos',5:'VPos'}
    df['label'] = df['label'].apply(lambda x: d[x])

    skf = model_selection.StratifiedKFold(n_splits = 5)

    for f, (t_,v_) in enumerate(skf.split(X=df,y=df.label.values)):
        df.loc[v_,"kfold"] = f

    self.dfs_train = [df[df['kfold']!=f] for f in range(0,5)]
    self.dfs_valid = [df[df['kfold']==f] for f in range(0,5)]
    
  def process(self,model):
    self.history = self.run_train(model)
    return {'summary':self.summary,'history':self.history}
  
  def get_history(self):
    return self.history  

  def prepare_data(self,i):
    mat_tr = np.matrix([x for x in  self.dfs_train[i]['embeddings'].apply(lambda x: json.loads(x))])
    mat_ts = np.matrix([x for x in  self.dfs_valid[i]['embeddings'].apply(lambda x: json.loads(x))])
    y_tr = self.dfs_train[i].label.values
    y_ts = self.dfs_valid[i].label.values
    x_tr = np.squeeze(np.asarray(mat_tr))
    x_ts = np.squeeze(np.asarray(mat_ts))
    x_tr = x_tr.reshape((x_tr.shape[0],x_tr.shape[1],1))
    x_ts = x_ts.reshape((x_ts.shape[0],x_ts.shape[1],1))
    encoded_Y_tr = self.encoder.fit_transform(y_tr)
    encoded_Y_ts = self.encoder.fit_transform(y_ts)
    y_tr = np_utils.to_categorical(encoded_Y_tr)
    y_ts = np_utils.to_categorical(encoded_Y_ts)
    x_tr = tf.convert_to_tensor(x_tr, np.float32)
    y_tr = tf.convert_to_tensor(y_tr, np.float32)
    x_ts = tf.convert_to_tensor(x_ts, np.float32)
    y_ts = tf.convert_to_tensor(y_ts, np.float32)
    # train_ds = tf.data.Dataset.from_tensor_slices((x_tr, y_tr))
    # # print(train_ds)
    # test_ds = tf.data.Dataset.from_tensor_slices((x_ts, y_ts))
    return x_tr,y_tr,x_ts,y_ts

  def get_summary(self):
    return self.summary


  def Conv1D_model(self,sh):
    with strategy.scope():
      model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(sh,1)),
        tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='sigmoid'),
        tf.keras.layers.Dense(256, activation='sigmoid'),
        tf.keras.layers.Dense(128, activation='sigmoid'),
        tf.keras.layers.Dense(64, activation='sigmoid'),
        tf.keras.layers.Dense(5, activation='softmax')        
        ])
      model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
      model.summary()
    return model

  def Conv1D_BiRNN_model(self,sh):
    with strategy.scope():
      model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(sh,1)),
        # tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        # tf.keras.layers.MaxPooling1D(pool_size=3),
        # tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        # tf.keras.layers.MaxPooling1D(pool_size=3),
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='sigmoid'),
        tf.keras.layers.Dense(256, activation='sigmoid'),
        tf.keras.layers.Dense(128, activation='sigmoid'),
        tf.keras.layers.Dense(64, activation='sigmoid'),
        tf.keras.layers.Dense(5, activation='softmax')        
        ])
      model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
      model.summary()
    return model
  
  def Conv1D_BiLSTM_model(self,sh):
    with strategy.scope():
      model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(sh,1)),
        # tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        # tf.keras.layers.MaxPooling1D(pool_size=3),
        # tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        # tf.keras.layers.MaxPooling1D(pool_size=3),
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='sigmoid'),
        tf.keras.layers.Dense(256, activation='sigmoid'),
        tf.keras.layers.Dense(128, activation='sigmoid'),
        tf.keras.layers.Dense(64, activation='sigmoid'),
        tf.keras.layers.Dense(5, activation='softmax')        
        ])
      model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
      model.summary()
    return model
  
  def Conv1D_BiGRU_model(self,sh):
    with strategy.scope():
      model = tf.keras.models.Sequential([
        # tf.keras.layers.Input(shape=(sh,1)),
        # tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        # tf.keras.layers.MaxPooling1D(pool_size=3),
        # tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        # tf.keras.layers.MaxPooling1D(pool_size=3),
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='sigmoid'),
        tf.keras.layers.Dense(256, activation='sigmoid'),
        tf.keras.layers.Dense(128, activation='sigmoid'),
        tf.keras.layers.Dense(64, activation='sigmoid'),
        tf.keras.layers.Dense(5, activation='softmax')        
        ])
      model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
      model.summary()
    return model

  def run_train(self,model):
    his = []
    for i in range(5):
      x_tr,y_tr,x_ts,y_ts = self.prepare_data(i)
      # print(train_ds.shape)
      # print(test_ds.shape)
      his.append(model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()]))
      # eval_loss, eval_acc = model.evaluate(x_ts,y_ts)
      # his.append((eval_loss,eval_acc))
      
    return his


In [ ]:
c = Conv1D()
c.read_dataset('/content/drive/MyDrive/sent_elec_690k.csv')

In [ ]:
c1d = c.Conv1D_BiLSTM_model(256)
x_c1d = c.process(c1d)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 256, 32)           2304      
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 3

In [ ]:
dfs = []
for i in range(len(x_c1d['history'])):
  df = pd.DataFrame.from_dict(x_c1d['history'][i].history)
  df['fold'] = i+1
  df['epoch'] = [i+1 for i in range(len(df))]
  dfs.append(df)

In [ ]:
df_fin = pd.concat(dfs,ignore_index=True)
df_fin.to_csv('/content/drive/MyDrive/sent_elec_BiLSTM.csv',index = False,header = True)

In [ ]:
acc = []
val_acc = []
loss = []
val_loss = []
for i in range(len(x['history'])):
  acc.append(x['history'][i].history['accuracy'][-1])
  loss.append(x['history'][i].history['loss'][-1])
  val_acc.append(x['history'][i].history['val_accuracy'][-1])
  val_loss.append(x['history'][i].history['val_loss'][-1])

NameError: ignored

In [ ]:
x['history'][i].history.items()

dict_items([('loss', [0.5154834389686584, 0.5145599842071533, 0.5148510336875916]), ('accuracy', [0.745518684387207, 0.746662437915802, 0.7463775277137756]), ('val_loss', [0.49613091349601746, 0.4944232702255249, 0.49505171179771423]), ('val_accuracy', [0.7572959065437317, 0.7606122493743896, 0.7589966058731079])])

In [ ]:
res = {'Conv1D accuracy': acc,'Conv1D loss':loss,'Conv1D val_loss':val_loss,'Conv1D val_accuracy':val_acc}

In [ ]:
res

{'Conv1D accuracy': [0.7419090270996094,
  0.7462712526321411,
  0.7448129057884216,
  0.7440263628959656,
  0.7463775277137756],
 'Conv1D loss': [0.5190023183822632,
  0.514137864112854,
  0.5159708261489868,
  0.5161250233650208,
  0.5148510336875916],
 'Conv1D val_accuracy': [0.7615816593170166,
  0.7594558000564575,
  0.7586564421653748,
  0.7592856884002686,
  0.7589966058731079],
 'Conv1D val_loss': [0.5027748346328735,
  0.5013351440429688,
  0.4960257112979889,
  0.4949069619178772,
  0.49505171179771423]}

In [ ]:
c = Conv1D()
c.read_dataset('/content/drive/MyDrive/sar_elec_294k.csv')
x = c.process(256)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 254, 32)           128       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 84, 32)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 82, 32)            3104      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 27, 32)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 27, 32)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 27, 32)            4800      
_________________________________________________________________
flatten_4 (Flatten)          (None, 864)              

In [ ]:
df = pd.DataFrame.from_dict(x['history'][0].history)

In [ ]:
df['epoch'] = [i+1 for i in range(len(df))]

In [ ]:
dfs = []
for i in range(len(x['history'])):
  df = pd.DataFrame.from_dict(x['history'][i].history)
  df['fold'] = i+1
  df['epoch'] = [i+1 for i in range(len(df))]
  dfs.append(df)

In [ ]:
df_fin = pd.concat(dfs,ignore_index=True)

In [ ]:
df_fin.to_csv('/content/drive/MyDrive/sar_elec_CONV-1D-2-BiGRU.csv',index = False,header = True)

In [ ]:
x2.shape

(58800, 256, 1)

In [ ]:
type(x1)

numpy.matrix

In [ ]:
z.shape

(235200, 256)

In [ ]:
y2.shape

(58800, 2)

In [ ]:
import pandas as pd
# df = pd.read_csv()

In [ ]:
df.columns

Index(['id', 'embeddings', 'label', 'kfold'], dtype='object')

In [ ]:
from collections import Counter
c = Counter(list(df.label.values))

In [ ]:
c.keys()

dict_keys(['sarcasm', 'normal'])

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(256,1)),
        tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        # tf.keras.layers.AveragePooling1D(pool_size=3, strides=1),
        tf.keras.layers.Conv1D(32,3, activation='relu',padding='valid'),
        # tf.keras.layers.AveragePooling1D(pool_size=3, strides=1),
      #   tf.keras.layers.Conv1D(32,3,activation='relu',padding='valid'),
      #   # tf.keras.layers.AveragePooling1D(pool_size=3, strides=1),
      #  tf.keras.layers.Conv1D(32,3,activation='relu',padding='valid'),
        # tf.keras.layers.AveragePooling1D(pool_size=3, strides=1),
        tf.keras.layers.MaxPooling1D(pool_size=3),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, dropout=0.2, recurrent_dropout=0.3)),
        # tf.keras.layers.Dropout(0.25),
        # tf.keras.layers.MaxPooling1D(pool_size=3),
        # tf.keras.layers.Dropout(0.25),
        # tf.keras.layers.AveragePooling1D(pool_size=3, strides=1),
        # tf.keras.layers.GlobalMaxPooling1D(),
        # tf.keras.layers.GRU(512,dropout=0.2,recurrent_dropout=0.3),
        # tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        # tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(512, activation='sigmoid'),
        # tf.keras.layers.Dense(512, activation='sigmoid'),
        # tf.keras.layers.Dense(256, activation='sigmoid'),
        tf.keras.layers.Dense(256, activation='sigmoid'),
        # tf.keras.layers.GlobalMaxPooling1D(),
        # tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(5, activation='softmax')
# # model.add(Dense(512, activation='sigmoid'))
# # model.add(Dropout(0.25))
        
])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 254, 32)           128       
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 252, 32)           3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 84, 32)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               591872    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_21 (Dense)             (None, 512)              